<a href="https://colab.research.google.com/github/hamingsi/potential-journey/blob/main/matrix_decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import numpy as np
from keras import Model
import keras.backend as K
from keras.layers import Embedding,Reshape,Input,Dot
from sklearn.model_selection import train_test_split



data = np.loadtxt('ratings.dat', encoding='utf-8',dtype=np.uint32,delimiter="::")

x_train,x_test,y_train,y_test = train_test_split(data[:,:2],data[:,2], random_state=42, test_size=0.2)

x_test,x_val,y_test,y_val = train_test_split(x_test,y_test, random_state=42, test_size=0.5)

print(len(x_train))
print(len(x_test))
print(len(x_val))

users = 6040
movies = 3952

800167
100021
100021


In [58]:
def Recmand_model(num_user, num_movie, k):
    # 将输入映射为向量
    # expected input[None,] None means not known the shape
    input_usr = Input(shape=[None,],dtype="int32")
    # Embedding(input_dim, output_dim) output=input_length*k weight=(num_user+1)*k
    # 输入值相当于索引 输出值为对应权重
    # P matrix 把输入的值编码为k维向量
    model_usr = Embedding(num_user+1, k, input_length=1)(input_usr)
    model_usr = Reshape((k,))(model_usr)

    input_movie = Input(shape=[None,],dtype="int32")
    # Q matrix
    model_movie = Embedding(num_movie+1, k, input_length=1)(input_movie)
    model_movie = Reshape((k,))(model_movie)
    # P[i]*Q[i]
    out = Dot(1)([model_usr,model_movie])
    # 需要保证之后调用Model的输入与这个[,]格式相同
    model = Model(inputs=[input_usr,input_movie], outputs=out)
    model.compile(loss='mse',optimizer='Adam') # 使用MSE作为损失函数 Adam作为优化器
    model.summary()
    return model

In [59]:
model = Recmand_model(users,movies,100)

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_10 (Embedding)       (None, None, 100)    604100      ['input_11[0][0]']               
                                                                                                  
 embedding_11 (Embedding)       (None, None, 100)    395300      ['input_12[0][0]']               
                                                                                            

In [60]:
train_x = [x_train[:,0],x_train[:,1]]
test_x = [x_test[:,0],x_test[:,1]]
val_x = [x_val[:,0],x_val[:,1]]

In [55]:
model.fit(train_x,y_train,batch_size = 100,validation_data=(val_x, y_val),epochs =10)

Epoch 1/10


8000/8002 [============================>.] - ETA: 0s - loss: 3.0297

8002/8002 [==============================] - 128s 16ms/step - loss: 3.0292 - val_loss: 0.8965
Epoch 2/10
8002/8002 [==============================] - 125s 16ms/step - loss: 0.8165 - val_loss: 0.8271
Epoch 3/10
8002/8002 [==============================] - 120s 15ms/step - loss: 0.7140 - val_loss: 0.8019
Epoch 4/10
8002/8002 [==============================] - 122s 15ms/step - loss: 0.6057 - val_loss: 0.8037
Epoch 5/10
8002/8002 [==============================] - 128s 16ms/step - loss: 0.4989 - val_loss: 0.8250
Epoch 6/10
8002/8002 [==============================] - 128s 16ms/step - loss: 0.4112 - val_loss: 0.8692
Epoch 7/10
8002/8002 [==============================] - 125s 16ms/step - loss: 0.3486 - val_loss: 0.9119
Epoch 8/10
8002/8002 [==============================] - 126s 16ms/step - loss: 0.3057 - val_loss: 0.9578
Epoch 9/10
8002/8002 [==============================] - 127s 16ms/step - loss: 0.2766 - val_loss: 0.9989
Epoch 10/10
8002/8002 [==============================] - 128s 16ms

In [56]:
results = model.evaluate(test_x, y_test, batch_size=128)
print("test loss:",results)

782/782 [==============================] - 2s 2ms/step - loss: 1.0326
test loss: 1.0326240062713623
